```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

Goal: Find the sum of all numbers that are adjacent to a symbol, including diagonals.
```
['#', '$', '%', '&', '*', '+', '-', '/', '=', '@']
    
```



In [61]:
from pprint import pprint
class Matcher(object):
    
    def __init__(self):
       self.load_file(test_mode=False)
       self.control_chars = ['#', '$', '%', '&', '*', '+', '-', '/', '=', '@']

       for i in range(0, 10):
            self.int_map[str(i)] = i
    
    int_map = {
        "zero": 0,
        "one": 1,
        "two": 2,
        "three": 3,
        "four": 4,
        "five": 5,
        "six": 6,
        "seven": 7,
        "eight": 8,
        "nine": 9
    }
    
    def get_int_mapping(self, s):
        """
            str:"one" -> int:1
            str:"1"   -> int:1
        """
        if s is None:
            return None
        
        for k,v in self.int_map.items():
            if k == s.lower():
                return v
        return None

    def load_file(self, test_mode=False):
        """
        Load file return array with all test inputs
        """
        file_name = "3.txt"
        if test_mode:
            file_name = str(file_name.split(".")[0]) + "_test.txt"

        with open("inputs/{}".format(file_name), "r") as f:
            l = f.read()
        self.input_arr = l.split("\n")

    @staticmethod
    def init_sequence():
        return {
            "start_pos": None,
            "end_pos": None,
            "line_no": None,
            "is_control_adjacent": False,
            "value": ""
        }
    
    @staticmethod 
    def validate_search_params(**kwargs):
        """
        Validate search parameters
        """
        if kwargs.get("line_no") < 0:
            raise Exception("Invalid value passed in, line_no cannot be less than 0.")
        if kwargs.get("start_pos") < 0:
            raise Exception("Invalid value passed in, start_pos cannot be less than 0.")
        if kwargs.get("end_pos") > kwargs.get("array_length"):
            raise Exception("Invalid value passed in, end_pos cannot be greater than the length of the line.")


    def check_for_control_alignment(self, full_array, line_no, start_pos, end_pos):
        """
        For a given sequence, denoted by a line number, start index, and end index.
        Determine if any control characters are adjacent to the sequence.
        Up, down, left, right, and diagonal.
        """
        Matcher.validate_search_params(line_no=line_no, start_pos=start_pos, end_pos=end_pos, array_length=len(full_array))

        for i in range(start_pos, end_pos+1):
            # print(full_array[line_no][i])
            
            if start_pos > 0:                                                   # Check left
                if full_array[line_no][start_pos-1] in self.control_chars:
                    return True
                
            if end_pos < len(full_array[line_no]):                              # Check right
                if full_array[line_no][end_pos+1] in self.control_chars:
                    return True
                
            if line_no > 0:                                                     # Check up
                if full_array[line_no-1][i] in self.control_chars:
                    return True
                
            if line_no < len(full_array)-1:                                     # Check down
                if full_array[line_no+1][i] in self.control_chars:
                    return True
                
            if line_no > 0 and start_pos > 0:                                   # Check up left
                if full_array[line_no-1][start_pos-1] in self.control_chars:
                    return True
            
            if line_no < len(full_array)-1 and start_pos > 0:                   # Check down left
                if full_array[line_no+1][start_pos-1] in self.control_chars:
                    return True
                
            if line_no > 0 and end_pos < len(full_array[line_no]):              # Check up right
                if full_array[line_no-1][end_pos+1] in self.control_chars:
                    return True
                
            if line_no < len(full_array)-1 and end_pos < len(full_array[line_no]):  # Check down right
                if full_array[line_no+1][end_pos+1] in self.control_chars:
                    return True


    def run(self):
        all_sequences = []
        unique = []
        for i in range(len(self.input_arr)):
            ii = self.input_arr[i]
            for c in ii:
                if c not in "abcdefghijklmnopqrstuvwxyz":
                    unique.append(c)

        print("unique: {}".format(sorted(list(set(unique)))))

        for i in range(len(self.input_arr)):
            line = self.input_arr[i]                                            # Get the current line
            # print(line)
            
            line_sequences = []                                                 # Array for all numerical sequences found on a line
            current_sequence = Matcher.init_sequence()                          # Datastructure for the current sequence
                                                                                # Includes start_pos, line_no, end_pos, value, etc
            for j in range(len(line)):                                          # For each character in the string
                c = line[j]
                if isinstance(self.get_int_mapping(c), int):                    # If it's a number
                    if current_sequence["start_pos"] is None:                   # If we haven't started a sequence, do so now
                        current_sequence["start_pos"] = j
                    current_sequence["line_no"] = i                             # Set the line number
                    current_sequence["value"] += c                              # Add the value to the sequence

                elif len(current_sequence.get("value")):                        # If it's not a number, and we have an existing sequence
                    current_sequence["end_pos"] = j - 1                         # Mark the sequences end
                    line_sequences.append(current_sequence)                     # And add it to lists, so we can post process for control adjacent chars
                    current_sequence = Matcher.init_sequence()

            if len(line_sequences):                                             # Done with this line, and we found sequences
                all_sequences.append(line_sequences)                            # Save all line sequences to a list

        aligned_sum = 0
        for line_sequences in all_sequences:                                    # We have a 2d array [line_number][seuqnce in that line]
            for s in line_sequences:                                            # For each sequence in a specific line
                try:
                    aligned = self.check_for_control_alignment(self.input_arr, 
                                                    s["line_no"], 
                                                    s["start_pos"], 
                                                    s["end_pos"]
                                                    )
                except IndexError as ex:
                    print("Caught exception: {}".format(ex))
                    print(s)
                    print(self.input_arr[s["line_no"]])
                    raise ex


                if aligned:
                    aligned_sum += int(s["value"])
        print("Aligned Sum: {}".format(aligned_sum))



m = Matcher()
m.run()

unique: ['#', '$', '%', '&', '*', '+', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', '@']
Aligned Sum: 521979
